In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd 
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
#Load the data into Pandas dataframe
df = pd.read_csv('task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))
df.head()

,class,tweet
0,0,"depression hurts, cymbalta can help"
1,0,"@jessicama20045 right, but cipro can make thin..."
2,0,@fibby1123 are you on paxil .. i need help
3,0,@redicine the lamotrigine and sjs just made ch...
4,0,have decided to skip my #humira shot today. my...


In [3]:
df = df.iloc[np.random.permutation(len(df))]
df.head()

,class,tweet
11534,0,just went to cuba usde cocky shun diarrhea gil...
1182,0,#camh #badpharma - causes - drug-induced #psyc...
13235,0,i had an advair inhaler for a cough. no asthma...
12012,0,cymbalta cialis interaction
482,0,@sto_plc baclofen 10mg out of stock and pharma...


In [4]:
X = df['tweet']
Y = df['class']

feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)
type(X_train), type(y_train)

(scipy.sparse.csr.csr_matrix, pandas.core.series.Series)

In [5]:
# class sklearn.svm.SVC(C=1.0, 
#                       kernel='rbf', 
#                       degree=3, 
#                       gamma='scale', 
#                       coef0=0.0, 
#                       shrinking=True, 
#                       probability=False, 
#                       tol=0.001, 
#                       cache_size=200, 
#                       class_weight=None, 
#                       verbose=False, 
#                       max_iter=-1, 
#                       decision_function_shape='ovr', 
#                       break_ties=False, 
#                       random_state=None)[source]

In [6]:
SVM = SVC(kernel='linear')
SVM.fit(X_train,y_train)

predictions_SVM = SVM.predict(X_test)

In [7]:
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100)
print("SVM F1 Score -> ", f1_score(predictions_SVM, y_test)*100)
print("SVM Roc_auc Score -> ", roc_auc_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  91.74981747383791
SVM F1 Score ->  27.71855010660981
SVM Roc_auc Score ->  85.63613293197744


# SHUFFLE SPLIT + Cross Val.

In [22]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, cross_val_predict
from scipy.sparse import hstack

data = df
data = data.iloc[np.random.permutation(len(df))]
data.head()

,class,tweet
8802,0,i've had some bad reactions with the gabapenti...
1219,0,"depression hurts, cymbalta will help"
6793,0,@danmallia_ i need alvesco but id be ok with a...
7671,0,@thedalyplanet xarelto commercial?
19033,0,to jesus through miralax


In [23]:
X = data['tweet']
Y = data['class']

feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)
type(X_train), type(y_train)

(scipy.sparse.csr.csr_matrix, pandas.core.series.Series)

In [34]:
SVM2 = SVC(kernel='linear', C=1) #.fit(X_train,y_train)
tols = [0.1, 0.01, 0.001, 0.0001]
best_score = 0
best_tol = 0
for tol in tols:
    SVM2 = SVC(kernel='linear', C=1, tol=tol)
    scores = cross_val_score(SVM2, X_train, y_train, cv=3, scoring='f1_macro')
    m = scores.mean()
    if m > best_score:
        best_score = m
        best_tol = tol
    print('tol =', tol, 'score =', scores.mean())
    
print('best_tol =', best_tol, 'best_score =', best_score)

tol = 0.1 score = 0.6098072651260243
tol = 0.01 score = 0.6092685461890435
tol = 0.001 score = 0.6097606193357457
tol = 0.0001 score = 0.6092685461890435
best_tol = 0.1 best_score = 0.6098072651260243


In [36]:
from sklearn.metrics import classification_report
SVM2 = SVC(kernel='linear', C=1, tol=best_tol).fit(X_train, y_train)
predictions_SVM2 = SVM2.predict(X_test)
print(classification_report(y_test, predictions_SVM2))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      3754
           1       0.81      0.23      0.35       355

    accuracy                           0.93      4109
   macro avg       0.87      0.61      0.66      4109
weighted avg       0.92      0.93      0.91      4109



In [9]:
# k = 6
# samp = len(data)//6

# training_data = data[:samp]
# control_data = data[samp:]

# #training_data, control_data

***Проблема с конкатенацией массивов...***

In [10]:
# split = ShuffleSplit(n_splits=5, test_size=1/4, random_state=5)

# for fold in range(k):
#     print(fold+1)

#     #X_train = np.append(X.toarray()[:samp*fold], X.toarray()[samp*(fold+1):])
#     X_train = np.append(data['tweet'][:samp*fold], data['tweet'][samp*(fold+1):])
#     X_test = X[samp*fold:samp*(fold+1)]
#     #y_train = np.append(np.array(Y[:samp*fold]), np.array(Y[samp*(fold+1):]))
#     y_train = np.append(data['tweet'][:samp*fold], data['tweet'][samp*(fold+1):])
#     y_test = Y[samp*fold:samp*(fold+1)]
    
#     SVM2 = SVC(kernel='linear')
#     SVM2.fit(X_train, y_train)
    
#     predictions_SVM2 = SVM2.predict(X_test)
    
#     f1.append(f1_score(predictions_SVM2, y_test)*100)
#     print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM2, y_test)*100)
#     print("SVM F1 Score -> ", f1[-1])
#     print("SVM Roc_auc Score -> ", roc_auc_score(predictions_SVM2, y_test)*100)
    
# predictions_SVM = SVM2.predict(X_test)